# Úkol č. 4 - regrese (do 2. ledna)

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných (ale celkem vyčištěných) datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdrojem dat je list *Data* v souboru `Residential-Building-Data-Set.xlsx` na course pages (originál zde: https://archive.ics.uci.edu/ml/datasets/Residential+Building+Data+Set#).
  * Popis datasetu najdete na listu *Descriptions* ve stejném souboru.
  

## Pokyny k vypracování

  1. Rozdělte data na trénovací a testovací množinu.
  1. Proveďte základní průzkum dat a příp. vyhoďte nezajímavé příznaky.
  1. Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `mean_absolute_error`.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.
  1. Použijte i jiný model než jen lineární a hřebenovou regresi.


## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte pouze tento Jupyter Notebook, opravujíví by neměl nic jiného potřebovat.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [1]:
# odtud už je to Vaše
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

Kazdou predikovanou hodnotu budeme predikovat v jednom modelu, tedy budou 2 modely, jeden pro predikci ceny prodeje, ktere nejlepe vystihuje prvnich 8 priznaku, druhy model pro stavebni cenu stavby z nejaktualnejsich dat, tedy z posledniho setu

In [7]:
data = pd.read_excel('Residential-Building-Data-Set.xlsx',list='Data',skiprows=1)

SalePrice_Y=data[['V-9']]       #   predikovana hodnota pro prvni model
CostPrice_Y=data[['V-10']]     #   predikovana hodnota pro druhy model

Nejdrive si vytvorime priznak celkovy pocet mesicu stavby a odstranime nepotrebne priznaky

In [8]:
data['V-30']=(data['COMPLETION YEAR']-data['START YEAR'])*4+(data['COMPLETION QUARTER']-data['START QUARTER'])  #   pocet ctvrtleti, delka doby stavby
#   ponechame pouze rok dokonceni
#   pocet vydanych stavebnich povoleni(V-11) mi prijde zbytecny priznak
#   odstranime i predikovane hodnoty
to_drop = ['START YEAR','COMPLETION QUARTER','START QUARTER','V-11']

#   data za prubeh 5ti let zprumerujeme, ostatni vyhodime
for i in range(18):
    x=11+i
    row = data[['V-'+str(x),'V-'+str(x)+'.1','V-'+str(x)+'.2','V-'+str(x)+'.3','V-'+str(x)+'.4']]
    data['V-'+str(x)] = row.mean(axis=1)

for i, _ in data.iteritems():
    if '.' in i:
        to_drop.append(i)

data = data.drop(columns=to_drop)
data = data.rename(columns={'COMPLETION YEAR':'V-0'})
display(data)

,V-0,V-1,V-2,V-3,V-4,V-5,V-6,V-7,V-8,V-12,...,V-23,V-24,V-25,V-26,V-27,V-28,V-29,V-9,V-10,V-30
0,85,1,3150.0,920.0,598.5,190,1010.84,16,1200,53.92,...,1755.000,8007.4,64.004,58.608,3402.236,29574.90,628132.9,2200,410,16
1,89,1,7600.0,1140.0,3040.0,400,963.81,23,2900,97.24,...,8647.788,8675.2,98.188,97.958,12544.810,31950.00,1188995.8,5000,1000,23
2,81,1,4800.0,840.0,480.0,100,689.84,15,630,38.96,...,1755.000,6161.8,42.166,34.992,1571.104,27119.60,524764.8,1200,170,15
3,73,1,685.0,202.0,13.7,20,459.54,4,140,11.14,...,1489.818,1528.8,10.458,9.278,522.700,18075.15,141542.6,165,30,4
4,90,1,3000.0,800.0,1230.0,410,631.91,13,5000,176.16,...,9272.228,9343.0,144.290,150.740,9808.600,34438.00,2318397.0,5500,700,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,86,20,1350.0,350.0,108.0,80,251.37,9,830,92.42,...,8548.072,8578.8,94.538,94.006,12233.302,29482.05,1067772.0,1100,150,9
368,82,20,600.0,150.0,36.0,60,299.55,6,570,55.48,...,2988.896,8005.8,66.168,60.886,3665.396,23479.00,648845.6,740,80,6
369,86,20,1900.0,430.0,285.0,150,364.41,7,640,111.38,...,8920.690,8940.2,107.640,108.684,11764.864,30216.90,1181856.2,860,220,7
370,84,20,510.0,160.0,30.6,60,245.28,9,790,69.60,...,8065.908,8118.6,79.512,76.556,6122.494,26394.75,833494.6,1100,110,9


Hledame teda priznaky, ktere by mohli ovlivnovat jednotlive predikce. Zkusime si to vyhodnotit podle koleracnich matic

In [23]:
def bestColumns(data):
    corrMatrix = data.corr().abs()
    # print(corrMatrix)
    #   hledame hodnoty blizke 1 nebo -1, (tedy absolutni hodnota je blizka 1) pro priznaky V-9 a V-10
    # print(corrMatrix['V-9'])
    # print(corrMatrix['V-10'])

    SalePrice_X_t = pd.DataFrame(corrMatrix['V-9'], columns=['V-9']).sort_values(by=['V-9'],ascending=False)
    CostPrice_X_t = pd.DataFrame(corrMatrix['V-10'], columns=['V-10']).sort_values(by=['V-10'],ascending=False)


    SalePrice_X_t = SalePrice_X_t.T.apply(lambda x: x>0.5).T
    CostPrice_X_t = CostPrice_X_t.T.apply(lambda x: x>0.5).T

    # display(SalePrice_X_t.T)
    # display(CostPrice_X_t.T)

    #   vyseparujeme si data z pudovnich pro trenovani a testovani
    cols = data.columns
    SalePrice_X = pd.DataFrame([])
    CostPrice_X = pd.DataFrame([])

    for col in cols:
        if col in SalePrice_X_t.T and SalePrice_X_t.T[col]['V-9'] == True and col!='V-9':
            SalePrice_X[col] = data[col]
        if col in CostPrice_X_t.T and CostPrice_X_t.T[col]['V-10'] == True and col!='V-10':
            CostPrice_X[col] = data[col]
    return CostPrice_X,SalePrice_X

CostPrice_X,SalePrice_X=bestColumns(data)
display(SalePrice_X)
display(CostPrice_X)

,V-0,V-4,V-5,V-8,V-12,V-13,V-15,V-16,V-17,V-18,V-19,V-21,V-22,V-23,V-24,V-25,V-26,V-27,V-29,V-10
0,85,598.5,190,1200,53.92,59.096,282103.760,3279.76,58.220,163.152,8317.420,766.140000,824.220,1755.000,8007.4,64.004,58.608,3402.236,628132.9,410
1,89,3040.0,400,2900,97.24,96.642,598494.440,3683.86,98.820,171.360,11689.872,1313.860000,1410.340,8647.788,8675.2,98.188,97.958,12544.810,1188995.8,1000
2,81,480.0,100,630,38.96,39.874,144653.380,1274.32,33.426,178.040,4043.492,581.181700,679.752,1755.000,6161.8,42.166,34.992,1571.104,524764.8,170
3,73,13.7,20,140,11.14,6.970,32818.082,332.08,11.728,31.706,336.080,178.930213,188.000,1489.818,1528.8,10.458,9.278,522.700,141542.6,30
4,90,1230.0,410,5000,176.16,148.156,1447981.540,9359.78,190.060,283.132,49351.100,2458.040000,2709.200,9272.228,9343.0,144.290,150.740,9808.600,2318397.0,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,86,108.0,80,830,92.42,92.246,558624.500,3610.10,95.800,179.820,10346.302,1248.680000,1350.940,8548.072,8578.8,94.538,94.006,12233.302,1067772.0,150
368,82,36.0,60,570,55.48,60.992,300146.800,3542.98,61.700,138.352,7015.740,785.740000,848.800,2988.896,8005.8,66.168,60.886,3665.396,648845.6,80
369,86,285.0,150,640,111.38,104.222,726637.040,4905.96,106.420,162.900,19188.546,1592.280000,1720.640,8920.690,8940.2,107.640,108.684,11764.864,1181856.2,220
370,84,30.6,60,790,69.60,75.746,411145.680,4727.44,81.400,217.620,9260.250,969.180000,1098.460,8065.908,8118.6,79.512,76.556,6122.494,833494.6,110


,V-0,V-4,V-5,V-8,V-12,V-13,V-15,V-16,V-17,V-18,...,V-21,V-22,V-23,V-24,V-25,V-26,V-27,V-28,V-29,V-9
0,85,598.5,190,1200,53.92,59.096,282103.760,3279.76,58.220,163.152,...,766.140000,824.220,1755.000,8007.4,64.004,58.608,3402.236,29574.90,628132.9,2200
1,89,3040.0,400,2900,97.24,96.642,598494.440,3683.86,98.820,171.360,...,1313.860000,1410.340,8647.788,8675.2,98.188,97.958,12544.810,31950.00,1188995.8,5000
2,81,480.0,100,630,38.96,39.874,144653.380,1274.32,33.426,178.040,...,581.181700,679.752,1755.000,6161.8,42.166,34.992,1571.104,27119.60,524764.8,1200
3,73,13.7,20,140,11.14,6.970,32818.082,332.08,11.728,31.706,...,178.930213,188.000,1489.818,1528.8,10.458,9.278,522.700,18075.15,141542.6,165
4,90,1230.0,410,5000,176.16,148.156,1447981.540,9359.78,190.060,283.132,...,2458.040000,2709.200,9272.228,9343.0,144.290,150.740,9808.600,34438.00,2318397.0,5500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,86,108.0,80,830,92.42,92.246,558624.500,3610.10,95.800,179.820,...,1248.680000,1350.940,8548.072,8578.8,94.538,94.006,12233.302,29482.05,1067772.0,1100
368,82,36.0,60,570,55.48,60.992,300146.800,3542.98,61.700,138.352,...,785.740000,848.800,2988.896,8005.8,66.168,60.886,3665.396,23479.00,648845.6,740
369,86,285.0,150,640,111.38,104.222,726637.040,4905.96,106.420,162.900,...,1592.280000,1720.640,8920.690,8940.2,107.640,108.684,11764.864,30216.90,1181856.2,860
370,84,30.6,60,790,69.60,75.746,411145.680,4727.44,81.400,217.620,...,969.180000,1098.460,8065.908,8118.6,79.512,76.556,6122.494,26394.75,833494.6,1100


Vidíme, že v obou případech je jedna predikovana hodnota závislá na té druhé a naopak. Můžeme buď odstranit z obou tabulek druhou predikovanou hodnotu, nebo muzeme odstranit pouze z jednoho s podminkou, ze ta nezavisla bude hodnocena prednostne. odstranim pouze zavislost v CostPrice na V-9

In [24]:
CostPrice_X = CostPrice_X.drop(columns=['V-9'])
# SalePrice_X = SalePrice_X.drop(columns=['V-10'])

vytvorime si funkce na rozdeleni dat a nejake vizualizace

In [25]:
def splitData(x_data,y_data):
    X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=42)
    print('Train X shape: ', X_train.shape)
    print('Test X shape: ', X_test.shape)
    return X_train, X_test, Y_train, Y_test

def visualization(data,linear):   
    plt.scatter(data, linear)
    plt.plot([0,50], [0,50], 'r')
    plt.show() 

Nyní si vytvoříme modely nejdříve pro nestandardizovaná data

In [26]:
#   CostPrice predikce
print("COST_PRICE --> V-10")
cp_x_tr,cp_x_test,cp_y_tr,cp_y_test = splitData(CostPrice_X,CostPrice_Y)

#   n- nestandardizovana, cp- costPrice, tr- train data, lr- linear regression, rr- ridge regression, fr- random forest regresion
n_cp_lr = LinearRegression()
n_cp_lr.fit(cp_x_tr,cp_y_tr)
n_cp_Yth = n_cp_lr.predict(cp_x_test)
print('linear regression MAE:', np.sqrt(mean_absolute_error(n_cp_Yth, np.array(cp_y_test))))

n_cp_rr = Ridge()
n_cp_rr.fit(cp_x_tr,cp_y_tr)
n_cp_Yth = n_cp_lr.predict(cp_x_test)
print('ridge regression MAE:', np.sqrt(mean_absolute_error(n_cp_Yth, np.array(cp_y_test))))

n_cp_fr = RandomForestRegressor()
n_cp_fr.fit(cp_x_tr,cp_y_tr)
n_cp_Yth = n_cp_lr.predict(cp_x_test)
print('random forest regression MAE:', np.sqrt(mean_absolute_error(n_cp_Yth, np.array(cp_y_test))))


print("\n------------------------------------------------------------------\n")
#   SalePrice predikce
print("SALE_PRICE --> V-9")
sp_x_tr,sp_x_test,sp_y_tr,sp_y_test = splitData(SalePrice_X,SalePrice_Y)

n_sp_lr = LinearRegression()
n_sp_lr.fit(sp_x_tr,sp_y_tr)
n_sp_Yth = n_sp_lr.predict(sp_x_test)
print('linear regression MAE:', np.sqrt(mean_absolute_error(n_sp_Yth, np.array(sp_y_test))))

n_sp_rr = Ridge()
n_sp_rr.fit(sp_x_tr,sp_y_tr)
n_sp_Yth = n_sp_lr.predict(sp_x_test)
print('ridge regression MAE:', np.sqrt(mean_absolute_error(n_sp_Yth, np.array(sp_y_test))))

n_sp_fr = RandomForestRegressor()
n_sp_fr.fit(sp_x_tr,sp_y_tr)
n_sp_Yth = n_sp_lr.predict(sp_x_test)
print('random forest regression MAE:', np.sqrt(mean_absolute_error(n_sp_Yth, np.array(sp_y_test))))

COST_PRICE --> V-10
Train X shape:  (279, 20)
Test X shape:  (93, 20)
linear regression MAE: 4.458157755225569
ridge regression MAE: 4.458157755225569
random forest regression MAE: 4.458157755225569

------------------------------------------------------------------

SALE_PRICE --> V-9
Train X shape:  (279, 20)
Test X shape:  (93, 20)
linear regression MAE: 11.380832283166784
ridge regression MAE: 11.380832283166784
random forest regression MAE: 11.380832283166784


zkusime data standardizovat pomoci standardScaler

In [27]:
scaler = StandardScaler()
scaler.fit(data)
s_data = pd.DataFrame(scaler.transform(data),columns=(data.columns))
# display(s_data)

s_CostPrice_X, s_SalePrice_X = bestColumns(s_data)
s_CostPrice_X = s_CostPrice_X.drop(columns=['V-9'])
# s_SalePrice_X = s_SalePrice_X.drop(columns=['V-10'])

s_SalePrice_Y=s_data[['V-9']]      
s_CostPrice_Y=s_data[['V-10']] 

A naucime nove modely na standardizovana data

In [28]:
#   CostPrice predikce
print("COST_PRICE --> V-10")
cp_x_tr,cp_x_test,cp_y_tr,cp_y_test = splitData(s_CostPrice_X,s_CostPrice_Y)

#   s- standardizovana, cp- costPrice, tr- train data, lr- linear regression, rr- ridge regression, fr- logistic regresion
s_cp_lr = LinearRegression()
s_cp_lr.fit(cp_x_tr,cp_y_tr)
s_cp_Yth = n_cp_lr.predict(cp_x_test)
print('linear regression MAE:', np.sqrt(mean_absolute_error(s_cp_Yth, np.array(cp_y_test))))

s_cp_rr = Ridge()
s_cp_rr.fit(cp_x_tr,cp_y_tr)
s_cp_Yth = n_cp_lr.predict(cp_x_test)
print('ridge regression MAE:', np.sqrt(mean_absolute_error(s_cp_Yth, np.array(cp_y_test))))

s_cp_fr = RandomForestRegressor()
s_cp_fr.fit(cp_x_tr,cp_y_tr)
s_cp_Yth = n_cp_lr.predict(cp_x_test)
print('random forest regression MAE:', np.sqrt(mean_absolute_error(s_cp_Yth, np.array(cp_y_test))))


print("\n------------------------------------------------------------------\n")
#   SalePrice predikce
print("SALE_PRICE --> V-9")
sp_x_tr,sp_x_test,sp_y_tr,sp_y_test = splitData(s_SalePrice_X,s_SalePrice_Y)

s_sp_lr = LinearRegression()
s_sp_lr.fit(sp_x_tr,sp_y_tr)
s_sp_Yth = n_sp_lr.predict(sp_x_test)
print('linear regression MAE:', np.sqrt(mean_absolute_error(s_sp_Yth, np.array(sp_y_test))))

s_sp_rr = Ridge()
s_sp_rr.fit(sp_x_tr,sp_y_tr)
s_sp_Yth = n_sp_lr.predict(sp_x_test)
print('ridge regression MAE:', np.sqrt(mean_absolute_error(s_sp_Yth, np.array(sp_y_test))))

s_sp_fr = RandomForestRegressor()
s_sp_fr.fit(sp_x_tr,sp_y_tr)
s_sp_Yth = n_sp_lr.predict(sp_x_test)
print('random forest regression MAE:', np.sqrt(mean_absolute_error(s_sp_Yth, np.array(sp_y_test))))

COST_PRICE --> V-10
Train X shape:  (279, 20)
Test X shape:  (93, 20)
linear regression MAE: 52.68953484287594
ridge regression MAE: 52.68953484287594
random forest regression MAE: 52.68953484287594

------------------------------------------------------------------

SALE_PRICE --> V-9
Train X shape:  (279, 20)
Test X shape:  (93, 20)
linear regression MAE: 15.184433734241912
ridge regression MAE: 15.184433734241912
random forest regression MAE: 15.184433734241912
